# Hyperparameter Tuning Your Own R Algorithm with Your Own Container in Amazon SageMaker


---

This notebook's CI test result for us-west-2 is as follows. CI test results in other regions can be found at the end of the notebook. 

![This us-west-2 badge failed to load. Check your device's internet connectivity, otherwise the service is currently unavailable](https://prod.us-west-2.tcx-beacon.docs.aws.dev/sagemaker-nb/us-west-2/r_examples|r_byo_r_algo_hpo|tune_r_bring_your_own.ipynb)

---

_**Using Amazon SageMaker's Hyperparameter Tuning with a customer Docker container and R algorithm**_

---

**Read before running the notebook:**
- This notebook has been updated to SageMaker v2.0
- Use Python3 kernel for this notebook.
- Dockerfile has been updated to use [Amazon ECR Public Gallery](https://docs.aws.amazon.com/AmazonECR/latest/public/public-gallery.html)


## Contents

1. [Background](#Background)
1. [Setup](#Setup)
  1. [Permissions](#Permissions)
1. [Code](#Code)
  1. [Publish](#Publish)
1. [Data](#Data)
1. [Tune](#Tune)
1. [HPO Analysis](#HPO-Analysis)
1. [Host](#Host)
1. [Predict](#Predict)
1. [(Optional) Clean-up](#(Optional)-Clean-up)
1. [Wrap-up](#Wrap-up)

---
## Background

R is a popular open source statistical programming language, with a lengthy history in Data Science and Machine Learning.  The breadth of algorithms available as R packages is impressive and fuels a diverse community of users.  In this example, we'll combine one of those algorithms ([Multivariate Adaptive Regression Splines](https://en.wikipedia.org/wiki/Multivariate_adaptive_regression_splines)) with SageMaker's hyperparameter tuning capabilities to build a simple model on the well-known [Iris dataset](https://en.wikipedia.org/wiki/Iris_flower_data_set). 


This notebook will focus mainly on the integration of hyperparameter tuning and a custom algorithm container, as well as hosting the tuned model and making inference using the endpoint. For more examples, please see this [notebook](https://github.com/awslabs/amazon-sagemaker-examples/blob/master/advanced_functionality/r_bring_your_own/r_bring_your_own.ipynb).

---
## Setup

_This notebook was created and tested on an ml.m4.xlarge notebook instance._

Let's start by specifying:

- The S3 bucket and prefix that you want to use for training and model data.  This should be within the same region as the notebook instance, training, and hosting.
- The IAM role arn used to give training and hosting access to your data. See the [documentation](https://docs.aws.amazon.com/sagemaker/latest/dg/using-identity-based-policies.html) for more details on creating these.  Note, if a role not associated with the current notebook instance, or more than one role is required for training and/or hosting, please replace `sagemaker.get_execution_role()` with a the appropriate full IAM role arn string(s).

In [1]:
import sagemaker

bucket = sagemaker.Session().default_bucket()
prefix = "sagemaker/DEMO-hpo-r-byo"

role = sagemaker.get_execution_role()

sagemaker.config INFO - Not applying SDK defaults from location: /etc/xdg/sagemaker/config.yaml
sagemaker.config INFO - Not applying SDK defaults from location: /home/ec2-user/.config/sagemaker/config.yaml


Now we'll import the libraries we'll need for the remainder of the notebook.

In [2]:
import os
import boto3
import sagemaker
import pandas as pd
from sagemaker.tuner import (
    IntegerParameter,
    CategoricalParameter,
    ContinuousParameter,
    HyperparameterTuner,
)

### Permissions

Running this notebook requires permissions in addition to the normal `SageMakerFullAccess` permissions. This is because we'll be creating a new repository in Amazon ECR. The easiest way to add these permissions is simply to add the managed policy `AmazonEC2ContainerRegistryFullAccess` to the role associated with your notebook instance. There's no need to restart your notebook instance when you do this, the new permissions will be available immediately.

---
## Code

For this example, we'll need 3 supporting code files.  We'll provide just a brief overview of what each one does.  See the full R bring your own notebook for more details.

- **Fit**: `mars.R` creates functions to train and serve our model.
- **Serve**: `plumber.R` uses the [plumber](https://www.rplumber.io/) package to create a lightweight HTTP server for processing requests in hosting.  Note the specific syntax, and see the plumber help docs for additional detail on more specialized use cases.
- **Dockerfile**:  This specifies the configuration for our docker container.  Smaller containers are preferred for Amazon SageMaker as they lead to faster spin up times in training and endpoint creation, so this container is kept minimal. This docker file starts with base R image, installs `plumber` and `mda` libraries and their dependecies, then adds `mars.R` and `plumber.R`, and finally sets `mars.R` to run as the entrypoint when launched.

    - **Upodate:** The updated dockerfile leverages public R-Base image from [Amazon Public ECR Gallery](https://aws.amazon.com/about-aws/whats-new/2020/12/announcing-amazon-ecr-public-and-amazon-ecr-public-gallery/) which has been available since December 2020. Feel free to read more about this public gallery and browse for public images at https://gallery.ecr.aws/.

### Publish
Now, to publish this container to ECR, we'll run the comands below.

This command will take several minutes to run the first time.

In [3]:
algorithm_name = "rmars"

In [4]:
%%sh

# The name of our algorithm
algorithm_name=rmars

#set -e # stop if anything fails
account=$(aws sts get-caller-identity --query Account --output text)

# Get the region defined in the current configuration (default to us-west-2 if none defined)
region=$(aws configure get region)
region=${region:-us-east-1}

fullname="${account}.dkr.ecr.${region}.amazonaws.com/${algorithm_name}:latest"

# Get the login command from ECR and execute it directly
aws ecr get-login-password --region ${region} | docker login --username AWS --password-stdin ${account}.dkr.ecr.${region}.amazonaws.com

# If the repository doesn't exist in ECR, create it.
aws ecr describe-repositories --repository-names "${algorithm_name}" > /dev/null 2>&1

if [ $? -ne 0 ]
then
    aws ecr create-repository --repository-name "${algorithm_name}" > /dev/null
fi

# Build the docker image locally with the image name and then push it to ECR
# with the full name.
docker build  -t ${algorithm_name} .
docker tag ${algorithm_name} ${fullname}

docker push ${fullname}

WARNING! Your password will be stored unencrypted in /home/ec2-user/.docker/config.json.
Configure a credential helper to remove this warning. See
https://docs.docker.com/engine/reference/commandline/login/#credentials-store



Login Succeeded
Sending build context to Docker daemon  77.31kB
Step 1/8 : FROM public.ecr.aws/docker/library/r-base:latest
 ---> 16511f39cdb4
Step 2/8 : MAINTAINER Amazon SageMaker Examples <amazon-sagemaker-examples@amazon.com>
 ---> Using cache
 ---> 0669b86a8779
Step 3/8 : RUN apt-get -y update && apt-get install -y --no-install-recommends     apt-utils     wget     r-base     r-base-dev     libcurl4-openssl-dev     apt-transport-https     libsodium-dev     ca-certificates
 ---> Using cache
 ---> 5ee2e6615c92
Step 4/8 : RUN apt-get install -y --no-install-recommends     libjs-bootstrap     libjs-jquery     r-api-4.0     r-cran-crayon     r-cran-httpuv     r-cran-jsonlite     r-cran-lifecycle     r-cran-magrittr     r-cran-mime     r-cran-promises     r-cran-r6     r-cran-sodium     r-cran-stringi     r-cran-swagger     r-cran-webutils     r-cran-plumber
 ---> Using cache
 ---> 505aee30120b
Step 5/8 : RUN apt-get install -y --no-install-recommends     libc6     libgfortran5     r-cr

---
## Data
For this illustrative example, we'll simply use `iris`.  This a classic, but small, dataset used to test supervised learning algorithms.  Typically, the goal is to predict one of three flower species based on various measurements of the flowers' attributes.  Further details can be found [here](https://archive.ics.uci.edu/ml/datasets/iris).
* **Source:** Dua, D. and Graff, C. (2019). UCI Machine Learning Repository [http://archive.ics.uci.edu/ml]. Irvine, CA: University of California, School of Information and Computer Science.

Let's split the data to train and test datasets (70% / 30%) and then copy the data to S3 so that SageMaker training can access it.

In [5]:
column_names = ["Sepal.Length", "Sepal.Width", "Petal.Length", "Petal.Width", "Species"]
data = pd.read_csv(
    "s3://sagemaker-sample-files/datasets/tabular/iris/iris.data",
    names=column_names,
)

/home/ec2-user/anaconda3/envs/python3/lib/python3.10/site-packages/fsspec/registry.py:279: UserWarning: Your installed version of s3fs is very old and known to cause
severe performance issues, see also https://github.com/dask/dask/issues/10276

To fix, you should specify a lower version bound on s3fs, or
update the current installation.

  warnings.warn(s3_msg)


In [6]:
# Train/test split, 70%-30%
train_data = data.sample(frac=0.7, random_state=42)
test_data = data.drop(train_data.index)
test_data.head()

,Sepal.Length,Sepal.Width,Petal.Length,Petal.Width,Species
1,4.9,3.0,1.4,0.2,Iris-setosa
2,4.7,3.2,1.3,0.2,Iris-setosa
6,4.6,3.4,1.4,0.3,Iris-setosa
14,5.8,4.0,1.2,0.2,Iris-setosa
20,5.4,3.4,1.7,0.2,Iris-setosa


In [7]:
# Write to csv
train_data.to_csv("iris_train.csv", index=False)
test_data.to_csv("iris_test.csv", index=False)

In [8]:
# write to S3
train_file = "iris_train.csv"
boto3.Session().resource("s3").Bucket(bucket).Object(
    os.path.join(prefix, "train", train_file)
).upload_file(train_file)

_Note: Although we could do preliminary data transformations in the notebook, we'll avoid doing so, instead choosing to do those transformations inside the container.  This is not typically the best practice for model efficiency, but provides some benefits in terms of flexibility._

---
## Tune

Now, let's setup the information needed to train a Multivariate Adaptive Regression Splines model on `iris` data.  In this case, we'll predict `Sepal.Length` rather than the more typical classification of `Species` in order to show how factors might be included in a model and to limit the use case to regression.

First, we'll get our region and account information so that we can point to the ECR container we just created.

In [9]:
region = boto3.Session().region_name
account = boto3.client("sts").get_caller_identity().get("Account")

Now we'll create an estimator using the [SageMaker Python SDK](https://github.com/aws/sagemaker-python-sdk).  This allows us to specify:
- The training container image in ECR
- The IAM role that controls permissions for accessing the S3 data and executing SageMaker functions
- Number and type of training instances
- S3 path for model artifacts to be output to
- Any hyperparameters that we want to have the same value across all training jobs during tuning

In [11]:
estimator = sagemaker.estimator.Estimator(
    image_uri="{}.dkr.ecr.{}.amazonaws.com/rmars:latest".format(account, region),
    role=role,
    instance_count=1,
    instance_type="ml.m5.xlarge",
    output_path="s3://{}/{}/output".format(bucket, prefix),
    sagemaker_session=sagemaker.Session(),
    hyperparameters={"degree": 2},
)  # Setting constant hyperparameter

# target is by defauld "Sepal.Length". See mars.R where this is set.

Once we've defined our estimator we can specify the hyperparameters that we'd like to tune and their possible values.  We have three different types of hyperparameters.
- Categorical parameters need to take one value from a discrete set.  We define this by passing the list of possible values to `CategoricalParameter(list)`
- Continuous parameters can take any real number value between the minimum and maximum value, defined by `ContinuousParameter(min, max)`
- Integer parameters can take any integer value between the minimum and maximum value, defined by `IntegerParameter(min, max)`

*Note, if possible, it's almost always best to specify a value as the least restrictive type.  For example, tuning `thresh` as a continuous value between 0.01 and 0.2 is likely to yield a better result than tuning as a categorical parameter with possible values of 0.01, 0.1, 0.15, or 0.2.*

In [12]:
# to set the degree as a varying HP to tune, use: 'degree': IntegerParameter(1, 3) and remove it from the Estimator

hyperparameter_ranges = {
    "thresh": ContinuousParameter(0.001, 0.01),
    "prune": CategoricalParameter(["TRUE", "FALSE"]),
}

Next we'll specify the objective metric that we'd like to tune and its definition.  This metric is output by a `print` statement in our `mars.R` file.  Its critical that the format aligns with the regular expression (Regex) we then specify to extract that metric from the CloudWatch logs of our training job.

In [13]:
objective_metric_name = "mse"
metric_definitions = [{"Name": "mse", "Regex": "mse: ([0-9\\.]+)"}]

Now, we'll create a `HyperparameterTuner` object, which we pass:
- The MXNet estimator we created above
- Our hyperparameter ranges
- Objective metric name and definition
- Whether we should maximize or minimize our objective metric (defaults to 'Maximize')
- Number of training jobs to run in total and how many training jobs should be run simultaneously.  More parallel jobs will finish tuning sooner, but may sacrifice accuracy.  We recommend you set the parallel jobs value to less than 10% of the total number of training jobs (we'll set it higher just for this example to keep it short).

In [14]:
tuner = HyperparameterTuner(
    estimator,
    objective_metric_name,
    hyperparameter_ranges,
    metric_definitions,
    objective_type="Minimize",
    max_jobs=9,
    max_parallel_jobs=3,
)

And finally, we can start our hyperparameter tuning job by calling `.fit()` and passing in the S3 paths to our train and test datasets.

*Note, typically for hyperparameter tuning, we'd want to specify both a training and validation (or test) dataset and optimize the objective metric from the validation dataset.  However, because `iris` is a very small dataset we'll skip the step of splitting into training and validation.  In practice, doing this could lead to a model that overfits to our training data and does not generalize well.*

In [15]:
tuner.fit({"train": "s3://{}/{}/train".format(bucket, prefix)}, wait=False)

No finished training job found associated with this estimator. Please make sure this estimator is only used for building workflow config
No finished training job found associated with this estimator. Please make sure this estimator is only used for building workflow config


Let's just run a quick check of the hyperparameter tuning jobs status to make sure it started successfully and is `InProgress`.

In [16]:
import time

status = boto3.client("sagemaker").describe_hyper_parameter_tuning_job(
    HyperParameterTuningJobName=tuner.latest_tuning_job.job_name
)["HyperParameterTuningJobStatus"]

while status != "Completed":
    status = boto3.client("sagemaker").describe_hyper_parameter_tuning_job(
        HyperParameterTuningJobName=tuner.latest_tuning_job.job_name
    )["HyperParameterTuningJobStatus"]

    completed = boto3.client("sagemaker").describe_hyper_parameter_tuning_job(
        HyperParameterTuningJobName=tuner.latest_tuning_job.job_name
    )["TrainingJobStatusCounters"]["Completed"]

    prog = boto3.client("sagemaker").describe_hyper_parameter_tuning_job(
        HyperParameterTuningJobName=tuner.latest_tuning_job.job_name
    )["TrainingJobStatusCounters"]["InProgress"]

    print(f"{status}, Completed Jobs: {completed}, In Progress Jobs: {prog}")

    time.sleep(30)

InProgress, Completed Jobs: 0, In Progress Jobs: 3
InProgress, Completed Jobs: 0, In Progress Jobs: 3
InProgress, Completed Jobs: 0, In Progress Jobs: 3
InProgress, Completed Jobs: 2, In Progress Jobs: 3
InProgress, Completed Jobs: 5, In Progress Jobs: 1
InProgress, Completed Jobs: 6, In Progress Jobs: 3
InProgress, Completed Jobs: 8, In Progress Jobs: 1
Completed, Completed Jobs: 9, In Progress Jobs: 0


Wait until the HPO job is complete, and then run the following cell:

In [17]:
boto3.client("sagemaker").describe_hyper_parameter_tuning_job(
    HyperParameterTuningJobName=tuner.latest_tuning_job.job_name
)["BestTrainingJob"]

{'TrainingJobName': 'rmars-240723-0437-003-28f16cd8',
 'TrainingJobArn': 'arn:aws:sagemaker:us-east-1:482851446821:training-job/rmars-240723-0437-003-28f16cd8',
 'CreationTime': datetime.datetime(2024, 7, 23, 4, 38, 7, tzinfo=tzlocal()),
 'TrainingStartTime': datetime.datetime(2024, 7, 23, 4, 38, 49, tzinfo=tzlocal()),
 'TrainingEndTime': datetime.datetime(2024, 7, 23, 4, 39, 48, tzinfo=tzlocal()),
 'TrainingJobStatus': 'Completed',
 'TunedHyperParameters': {'prune': 'FALSE', 'thresh': '0.004049633760804366'},
 'FinalHyperParameterTuningJobObjectiveMetric': {'MetricName': 'mse',
  'Value': 6.68666934967041},
 'ObjectiveStatus': 'Succeeded'}

---

## HPO Analysis

Now that we've started our hyperparameter tuning job, it will run in the background and we can close this notebook.  Once finished, we can use the [HPO Analysis notebook](https://github.com/awslabs/amazon-sagemaker-examples/tree/master/hyperparameter_tuning/analyze_results/HPO_Analyze_TuningJob_Results.ipynb) to determine which set of hyperparameters worked best.

For more detail on Amazon SageMaker's Hyperparameter Tuning, please refer to the AWS documentation. 

---
## Host

Hosting the model we just tuned takes three steps in Amazon SageMaker.  First, we define the model we want to host, pointing the service to the model artifact our training job just wrote to S3.

We will use the results of the HPO for this purpose, but using `hyper_parameter_tuning_job` method.

In [18]:
best_training = boto3.client("sagemaker").describe_hyper_parameter_tuning_job(
    HyperParameterTuningJobName=tuner.latest_tuning_job.job_name
)["BestTrainingJob"]

In [19]:
# Get the best trainig job and S3 location for the model file
best_model_s3 = boto3.client("sagemaker").describe_training_job(
    TrainingJobName=best_training["TrainingJobName"]
)["ModelArtifacts"]["S3ModelArtifacts"]
best_model_s3

's3://sagemaker-us-east-1-482851446821/sagemaker/DEMO-hpo-r-byo/output/rmars-240723-0437-003-28f16cd8/output/model.tar.gz'

In [20]:
import time

r_job = "DEMO-r-byo-" + time.strftime("%Y-%m-%d-%H-%M-%S", time.gmtime())

In [21]:
r_hosting_container = {
    "Image": "{}.dkr.ecr.{}.amazonaws.com/{}:latest".format(account, region, algorithm_name),
    "ModelDataUrl": best_model_s3,
}

create_model_response = boto3.client("sagemaker").create_model(
    ModelName=r_job, ExecutionRoleArn=role, PrimaryContainer=r_hosting_container
)

print(create_model_response["ModelArn"])

arn:aws:sagemaker:us-east-1:482851446821:model/DEMO-r-byo-2024-07-23-04-45-20


Next, let's create an endpoing configuration, passing in the model we just registered.  In this case, we'll only use a few c4.xlarges.

In [22]:
r_endpoint_config = "DEMO-r-byo-config-" + time.strftime("%Y-%m-%d-%H-%M-%S", time.gmtime())
print(r_endpoint_config)

create_endpoint_config_response = boto3.client("sagemaker").create_endpoint_config(
    EndpointConfigName=r_endpoint_config,
    ProductionVariants=[
        {
            "InstanceType": "ml.t2.medium",
            "InitialInstanceCount": 1,
            "ModelName": r_job,
            "VariantName": "AllTraffic",
        }
    ],
)

print("Endpoint Config Arn: " + create_endpoint_config_response["EndpointConfigArn"])

DEMO-r-byo-config-2024-07-23-04-45-21
Endpoint Config Arn: arn:aws:sagemaker:us-east-1:482851446821:endpoint-config/DEMO-r-byo-config-2024-07-23-04-45-21


Finally, we'll create the endpoints using our endpoint configuration from the last step.

In [23]:
%%time

r_endpoint = "DEMO-r-endpoint-" + time.strftime("%Y%m%d%H%M", time.gmtime())
print(r_endpoint)
create_endpoint_response = boto3.client("sagemaker").create_endpoint(
    EndpointName=r_endpoint, EndpointConfigName=r_endpoint_config
)
print(create_endpoint_response["EndpointArn"])

resp = boto3.client("sagemaker").describe_endpoint(EndpointName=r_endpoint)
status = resp["EndpointStatus"]
print("Status: " + status)

try:
    boto3.client("sagemaker").get_waiter("endpoint_in_service").wait(EndpointName=r_endpoint)
finally:
    resp = boto3.client("sagemaker").describe_endpoint(EndpointName=r_endpoint)
    status = resp["EndpointStatus"]
    print("Arn: " + resp["EndpointArn"])
    print("Status: " + status)

    if status != "InService":
        raise Exception("Endpoint creation did not succeed")

DEMO-r-endpoint-202407230445
arn:aws:sagemaker:us-east-1:482851446821:endpoint/DEMO-r-endpoint-202407230445
Status: Creating
Arn: arn:aws:sagemaker:us-east-1:482851446821:endpoint/DEMO-r-endpoint-202407230445
Status: InService
CPU times: user 107 ms, sys: 12.7 ms, total: 120 ms
Wall time: 4min 1s


---
## Predict
To confirm our endpoints are working properly, let's try to invoke the endpoint.

_Note: The payload we're passing in the request is a CSV string with a header record, followed by multiple new lines.  It also contains text columns, which the serving code converts to the set of indicator variables needed for our model predictions.  Again, this is not a best practice for highly optimized code, however, it showcases the flexibility of bringing your own algorithm._

In [24]:
import pandas as pd
import json

iris_test = pd.read_csv("iris_test.csv")

runtime = boto3.Session().client("runtime.sagemaker")

In [25]:
%%time

# there is a limit of max 500 samples at a time for invoking endpoints
payload = iris_test.drop(["Sepal.Length"], axis=1).to_csv(index=False)

response = runtime.invoke_endpoint(EndpointName=r_endpoint, ContentType="text/csv", Body=payload)

result = json.loads(response["Body"].read().decode())
display(result)

['4.63324327982493,4.80044511788773,4.9863289466641,5.23172030182186,5.05858590531148,5.24070314228686,4.76937655218207,3.55288534714371,5.27953163889477,5.17620611454173,6.1318693029285,6.25071620311115,6.12149400345727,5.14424047777516,6.0997711065632,5.69949288189905,5.99805512192479,6.16123806202157,5.79493080854062,5.96713066520703,5.40603582289417,5.9410692580294,5.91130138610351,5.98144578513731,6.12231111525494,5.72105106121319,5.84610602250233,7.17025319136242,6.94315420341912,6.47572230628897,5.72822541764846,7.21237615541374,6.51631007231049,6.5473915546383,6.48360200083876,6.07047328441353,6.75027496362148,6.08287598299863,6.6689680481329,6.94943555116772,6.59565735351025,6.75298292199343,6.67782045127373,6.40339216676571,6.34758171409982']

CPU times: user 14.8 ms, sys: 3.62 ms, total: 18.5 ms
Wall time: 172 ms


We can see the result is a CSV of predictions for our target variable.  Let's compare them to the actuals to see how our model did.

In [ ]:
import matplotlib.pyplot as plt
import numpy as np


plt.scatter(iris_test["Sepal.Length"], np.fromstring(result[0], sep=","), alpha=0.4, s=50)
plt.xlabel("Sepal Length(Actual)")
plt.ylabel("Sepal Length(Prediction)")
x = np.linspace(*plt.xlim())
plt.plot(x, x, linestyle="--", color="g", linewidth=1)
plt.xlim(4, 8)
plt.ylim(4, 8)

plt.show()

### (Optional) Clean-up

If you're ready to be done with this notebook, please run the cell below.  This will remove the hosted endpoint you created and avoid any charges from a stray instance being left on.

In [ ]:
boto3.client("sagemaker").delete_endpoint(EndpointName=r_endpoint)

## Notebook CI Test Results

This notebook was tested in multiple regions. The test results are as follows, except for us-west-2 which is shown at the top of the notebook.

![This us-east-1 badge failed to load. Check your device's internet connectivity, otherwise the service is currently unavailable](https://prod.us-west-2.tcx-beacon.docs.aws.dev/sagemaker-nb/us-east-1/r_examples|r_byo_r_algo_hpo|tune_r_bring_your_own.ipynb)

![This us-east-2 badge failed to load. Check your device's internet connectivity, otherwise the service is currently unavailable](https://prod.us-west-2.tcx-beacon.docs.aws.dev/sagemaker-nb/us-east-2/r_examples|r_byo_r_algo_hpo|tune_r_bring_your_own.ipynb)

![This us-west-1 badge failed to load. Check your device's internet connectivity, otherwise the service is currently unavailable](https://prod.us-west-2.tcx-beacon.docs.aws.dev/sagemaker-nb/us-west-1/r_examples|r_byo_r_algo_hpo|tune_r_bring_your_own.ipynb)

![This ca-central-1 badge failed to load. Check your device's internet connectivity, otherwise the service is currently unavailable](https://prod.us-west-2.tcx-beacon.docs.aws.dev/sagemaker-nb/ca-central-1/r_examples|r_byo_r_algo_hpo|tune_r_bring_your_own.ipynb)

![This sa-east-1 badge failed to load. Check your device's internet connectivity, otherwise the service is currently unavailable](https://prod.us-west-2.tcx-beacon.docs.aws.dev/sagemaker-nb/sa-east-1/r_examples|r_byo_r_algo_hpo|tune_r_bring_your_own.ipynb)

![This eu-west-1 badge failed to load. Check your device's internet connectivity, otherwise the service is currently unavailable](https://prod.us-west-2.tcx-beacon.docs.aws.dev/sagemaker-nb/eu-west-1/r_examples|r_byo_r_algo_hpo|tune_r_bring_your_own.ipynb)

![This eu-west-2 badge failed to load. Check your device's internet connectivity, otherwise the service is currently unavailable](https://prod.us-west-2.tcx-beacon.docs.aws.dev/sagemaker-nb/eu-west-2/r_examples|r_byo_r_algo_hpo|tune_r_bring_your_own.ipynb)

![This eu-west-3 badge failed to load. Check your device's internet connectivity, otherwise the service is currently unavailable](https://prod.us-west-2.tcx-beacon.docs.aws.dev/sagemaker-nb/eu-west-3/r_examples|r_byo_r_algo_hpo|tune_r_bring_your_own.ipynb)

![This eu-central-1 badge failed to load. Check your device's internet connectivity, otherwise the service is currently unavailable](https://prod.us-west-2.tcx-beacon.docs.aws.dev/sagemaker-nb/eu-central-1/r_examples|r_byo_r_algo_hpo|tune_r_bring_your_own.ipynb)

![This eu-north-1 badge failed to load. Check your device's internet connectivity, otherwise the service is currently unavailable](https://prod.us-west-2.tcx-beacon.docs.aws.dev/sagemaker-nb/eu-north-1/r_examples|r_byo_r_algo_hpo|tune_r_bring_your_own.ipynb)

![This ap-southeast-1 badge failed to load. Check your device's internet connectivity, otherwise the service is currently unavailable](https://prod.us-west-2.tcx-beacon.docs.aws.dev/sagemaker-nb/ap-southeast-1/r_examples|r_byo_r_algo_hpo|tune_r_bring_your_own.ipynb)

![This ap-southeast-2 badge failed to load. Check your device's internet connectivity, otherwise the service is currently unavailable](https://prod.us-west-2.tcx-beacon.docs.aws.dev/sagemaker-nb/ap-southeast-2/r_examples|r_byo_r_algo_hpo|tune_r_bring_your_own.ipynb)

![This ap-northeast-1 badge failed to load. Check your device's internet connectivity, otherwise the service is currently unavailable](https://prod.us-west-2.tcx-beacon.docs.aws.dev/sagemaker-nb/ap-northeast-1/r_examples|r_byo_r_algo_hpo|tune_r_bring_your_own.ipynb)

![This ap-northeast-2 badge failed to load. Check your device's internet connectivity, otherwise the service is currently unavailable](https://prod.us-west-2.tcx-beacon.docs.aws.dev/sagemaker-nb/ap-northeast-2/r_examples|r_byo_r_algo_hpo|tune_r_bring_your_own.ipynb)

![This ap-south-1 badge failed to load. Check your device's internet connectivity, otherwise the service is currently unavailable](https://prod.us-west-2.tcx-beacon.docs.aws.dev/sagemaker-nb/ap-south-1/r_examples|r_byo_r_algo_hpo|tune_r_bring_your_own.ipynb)
